# CNN for Brain Tumor Detection based on GLCM

## Import the Dataset and Libraries
In this section we need to get the dataset. The dataset is available on a github repository. The repository contains images of brain tumor and normal brain images. The author has also performed augmentation on the dataset, and it is also provided. We use the augmented images for our training purpose.
<br>Once the repository is cloned, we will import all the required libraries that we need for our problem

In [ ]:
!git  clone https://github.com/MohamedAliHabib/Brain-Tumor-Detection

Cloning into 'Brain-Tumor-Detection'...
remote: Enumerating objects: 2363, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2363 (delta 8), reused 4 (delta 4), pack-reused 2352
Receiving objects: 100% (2363/2363), 43.21 MiB | 26.77 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
# Import necessary libraries
import numpy as np
import os
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt  # Import plt module for visualization
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from skimage.transform import resize
from skimage.feature import graycoprops, graycomatrix


## Declaring Variables

In [ ]:
# Set directory path and parameters
data_dir = '/content/Brain-Tumor-Detection/augmented data'
categories = ['yes', 'no']  # Names of the categories
img_size = 256  # Size of the image after resizing
num_classes = len(categories)
batch_size = 16



## Feature Extraction
Here will define a function named extract_features(). this method takes image path as input and returns the features for each image. Alongwith the feature extraction, the preprocessing on the images is done. <br>
In this method, we calculate the GLCM features based on contrast, dissimilarity, homogeneity, ASM,and energy.

In [ ]:
# Extract GLCM features
def extract_features(image_path):
    # Load image and resize
    img = plt.imread(image_path)
    img = resize(img, (img_size, img_size), anti_aliasing=True)
    
    # Convert to grayscale
    img_gray = rgb2gray(img)
    
    # Calculate GLCM
    glcm = graycomatrix((img_gray * 255).astype('uint8'), distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    
    # Extract GLCM properties
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    ASM = graycoprops(glcm, 'ASM')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    
    # Return features as an array
    features = np.array([contrast, dissimilarity, homogeneity, ASM, energy, correlation])
    return features


## Extracting Features and Preparing for Training
In this step, we load each image, and calculate the features using the method defined earlier. <br>
Finally, the features are split in train/val categories for training and testing

In [ ]:
# Load data and extract features
X = []
y = []
for category in categories:
    path = os.path.join(data_dir, category)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        features = extract_features(img_path)
        X.append(features)
        y.append(categories.index(category))

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



## Model Definition
Here, an example CNN is defined for the explaination purposes. And, this CNN can further be improved for attaining more accuracy.

In [ ]:
# Build CNN model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=6))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


## Model Training
In this step, we train the model. We run the training for 10 epochs total. The hyperparameters (batch_size and epochs) can be changed for attaining better results.

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
104/104 [==============================] - 2s 6ms/step - loss: 8.0001 - accuracy: 0.5054 - val_loss: 1.3477 - val_accuracy: 0.5182
Epoch 2/10
104/104 [==============================] - 0s 2ms/step - loss: 2.5432 - accuracy: 0.5073 - val_loss: 3.0371 - val_accuracy: 0.4818
Epoch 3/10
104/104 [==============================] - 0s 3ms/step - loss: 2.2370 - accuracy: 0.5363 - val_loss: 3.6486 - val_accuracy: 0.4818
Epoch 4/10
104/104 [==============================] - 0s 2ms/step - loss: 2.1919 - accuracy: 0.5224 - val_loss: 1.0930 - val_accuracy: 0.5182
Epoch 5/10
104/104 [==============================] - 0s 3ms/step - loss: 2.1343 - accuracy: 0.5030 - val_loss: 5.9019 - val_accuracy: 0.4818
Epoch 6/10
104/104 [==============================] - 0s 2ms/step - loss: 2.3902 - accuracy: 0.5115 - val_loss: 10.9191 - val_accuracy: 0.4818
Epoch 7/10
104/104 [==============================] - 0s 2ms/step - loss: 3.7719 - accuracy: 0.5073 - val_loss: 2.1524 - val_accuracy: 0.4818
Epoch

## Model Testing
Now the accuracy and loss is calculated for our model. Here, val dataset is used, we can devide our dataset into three categories, train, test, and val as well. 

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_val, y_val)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

13/13 [==============================] - 0s 2ms/step - loss: 0.6771 - accuracy: 0.5956
Test loss: 0.6770885586738586
Test accuracy: 0.5956416726112366
